# 1. Install dependancies.

In [1]:
!pip install pandas pandarallel tqdm ipynb ipywidgets==7.7.2

# 2. Import and working directory

In [3]:
import pandas as pd
import tqdm.notebook as tqdm
from tqdm import tqdm as tq
import glob
import pathlib
import os
import re
import numpy as np
import subprocess
tq.pandas()

from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True, nb_workers=6) #6 process, each will take 8 cores, which leave me with 48 other cores (96 cores in total)

#Regex for best mode energy
regex_best_energy = re.compile(r"^1 +(-?[0-9]+\.[0-9]+) +0\.000 +0.000 +")



INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


Go to your working directory

In [4]:
os.chdir("/home/thibault/work/projects/other/Sylvie/SMINA")

# 3. Searching for pdbqt files

This will search all `pdbqt` files in `ligands` folder and put them in a pandas dataframe (used for quick parallelisation)

In [5]:
ligand_list = glob.glob("ligands/*.pdbqt")
df = pd.DataFrame(ligand_list, columns=["LigandFile"])

# Run processess

In [9]:
def run_smina(row):
    file=row.iloc[0]
    basename = pathlib.Path(file).stem
    outfolder = f"all_poses/{basename}"
    if not os.path.exists(outfolder):
        os.makedirs(outfolder)

    outputposes = f"{outfolder}/{basename}.pdbqt"
    outputlog = f"{outfolder}/{basename}.log"

    #default success = False. Only change it when it worked.
    success = False
    best_energy = np.nan
    #Check if the logfile already exist
    #if os.path.exists(outputlog):
    if False:
        with open(outputlog, "r") as log:
            stdout = log.readlines()
        success = True
    else:
        results = subprocess.run([
            "./smina.static", 
            "--config","ACDC_B.inp",
            "--ligand", file, 
            "--out", outputposes, 
            "--log",outputlog,
            "--cpu","8", 
            "--scoring","vinardo"],
            capture_output=True)
        if results.returncode == 0:
            success = True
        if success == True:
            stdout = results.stdout.decode("utf-8").split("\n")
        

    if success == True:
        #Get the best energy
        for line in stdout:
            match = regex_best_energy.findall(line)
            if match:
                best_energy=float(match[0])
                break
            else:
                best_energy = np.nan


    return pd.Series(
            {
            "name":basename,
            "filename":file,
            "success":success,
            "BestEnergy":best_energy
            }
        )
results = df.parallel_apply(lambda x: run_smina(x), axis=1)

results.to_csv("results.csv",sep=";")

In [36]:
results.query("BestEnergy < -11")

,name,filename,success,BestEnergy
827,TCMDC-138610,ligands/TCMDC-138610.pdbqt,True,-11.2
1224,TCMDC-137673,ligands/TCMDC-137673.pdbqt,True,-11.5
2419,TCMDC-139868,ligands/TCMDC-139868.pdbqt,True,-11.5
3011,TCMDC-125666,ligands/TCMDC-125666.pdbqt,True,-11.8
3026,TCMDC-139069,ligands/TCMDC-139069.pdbqt,True,-11.1
3620,TCMDC-124223,ligands/TCMDC-124223.pdbqt,True,-11.6
3945,TCMDC-138256,ligands/TCMDC-138256.pdbqt,True,-11.1
4734,TCMDC-139888,ligands/TCMDC-139888.pdbqt,True,-11.5
4848,TCMDC-140353,ligands/TCMDC-140353.pdbqt,True,-11.1
5002,TCMDC-125639,ligands/TCMDC-125639.pdbqt,True,-11.4


# Reformating the pdbqt to be "VINA LIKE" and extract the best pose

Vina pdbqt have a line with the result in the format `REMARK VINA X X X`.  
This code will add it by parsing the logfiles.

In [70]:

def reformat_and_extract_best_pose(row):
    file=row.iloc[0]
    basename = pathlib.Path(file).stem
    outfolder = f"all_poses/{basename}"
    outputposes = f"{outfolder}/{basename}.pdbqt"
    outputlog = f"{outfolder}/{basename}.log"
    regex_results = re.compile(r"^([0-9]+) +(-?[0-9]+\.[0-9]) +(-?[0-9]+\.[0-9]+) +(-?[0-9]+\.[0-9]+)")

    #default success = False. Only change it when it worked.
    success = False
    best_energy = np.nan
    #Check if the logfile already exist
    if os.path.exists(outputlog):
        with open(outputlog, "r") as log:
            stdout = log.readlines()
        success = True
        

    if success == True:
        #Get the best energy4
        scores = {}
        for line in stdout:
            match = regex_results.findall(line)
            if len(match)>0:
                model=match[0][0]
                energy=float(match[0][1])
                rmsdLB=float(match[0][2])
                rmsdUB=float(match[0][3])
                scores[model]=(energy, rmsdLB, rmsdUB)

    if success:
        newpdbqt = []
        best_model = []
        with open(outputposes,"r") as pdbqt:
            lines = pdbqt.readlines()
            model=0
            vinaLineAdded = False
            for line in lines:
                newpdbqt.append(line)
                if line.startswith("MODEL"):
                    model = line.strip().split(" ")[-1]
                    energy = float(scores[model][0])
                    rmsdLB = float(scores[model][1])
                    rmsdUB = float(scores[model][2])
                    VINALINE = f"REMARK VINA RESULT: {energy:>10.1f} {rmsdLB:>10.3f} {rmsdUB:>10.3f}\n"
                    newpdbqt.append(VINALINE)
                
                if model == "1":
                    best_model.append(line)
                    if vinaLineAdded == False:
                        best_model.append(VINALINE)
                        vinaLineAdded = True
                    

        with open(f"{outfolder}/{basename}_vinaFormat.pdbqt",'w') as vinaout:
            for line in newpdbqt:
                vinaout.write(line)

        with open(f"{outfolder}/{basename}_bestpose.pdbqt",'w') as bestout:
            for line in best_model:
                bestout.write(line)

_ = df.progress_apply(lambda x: reformat_and_extract_best_pose(x), axis=1)

100%|██████████| 13530/13530 [00:06<00:00, 1953.88it/s]
